# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv` . Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib

# TODO: Agregar las librerías que hagan falta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

in_colab = 'google.colab' in sys.modules

In [2]:
def get_metrics(y_true, y_pred):
    """Devuelve un DataFrame con accuracy, precision, recall y f1-score
    """
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    df = pd.DataFrame(
        data=[[accuracy, precision, recall, f1]],
        columns=['Accuracy', 'Precision', 'Recall', 'F1-Score']
    ).applymap("{0:.2%}".format)
    return df

In [3]:
def get_confusion_matrix(y_true, y_pred):
    """Devuelve un DataFrame con la matriz de confusión
    """
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1]).transpose()
    df = pd.DataFrame(
        cm, 
        columns=['Actual - Repaid', 'Actual - Default'],
        index=['Predicted - Repaid', 'Predicted - Default']
    )
    df.loc['Total']= df.sum()  # Agrego fila de totales
    df['Total'] = df.sum(axis=1)  # Agrego columna de totales
    
    return df

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [4]:
if in_colab:
  data_dir = "https://raw.githubusercontent.com/DiploDatos/IntroduccionAprendizajeAutomatico/master/data/loan_data.csv"
else:
  data_dir = "./data/loan_data.csv"

dataset = pd.read_csv(data_dir, comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [5]:
dataset.describe()

,TARGET,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
count,1854.000000,1854.000000,1854.000000,1854.000000,1854.000000,1854.000000,1854.000000,1854.000000,1854.000000,1854.000000,1854.000000
mean,0.166667,19111.758360,76316.051780,107321.088457,8.900216,0.187702,0.319849,180.300783,1.128910,21.857066,34.573408
std,0.372779,11000.345961,46227.026585,56039.685066,7.552677,0.704882,0.928452,84.838308,1.664555,9.510840,9.308794
min,0.000000,1700.000000,5627.000000,21144.000000,0.000000,0.000000,0.000000,0.486711,0.000000,0.000000,0.838118
25%,0.000000,12000.000000,48984.750000,70787.250000,3.000000,0.000000,0.000000,116.970718,0.000000,16.000000,29.427210
50%,0.000000,17000.000000,67201.000000,94198.000000,7.000000,0.000000,0.000000,174.967815,1.000000,21.000000,35.363407
75%,0.000000,23900.000000,93731.500000,122976.250000,13.000000,0.000000,0.000000,232.261800,2.000000,27.000000,39.357987
max,1.000000,89800.000000,399412.000000,512650.000000,41.000000,10.000000,10.000000,1168.233561,13.000000,65.000000,144.189001


In [6]:
dataset.sample(10)

,TARGET,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
1725,1,18200,67809.0,89339.0,0.0,3.0,0.0,102.460986,2.0,19.0,38.345341
1423,0,81400,21041.0,111304.0,26.0,0.0,0.0,146.322783,0.0,14.0,32.131373
1743,1,19300,96454.0,157809.0,3.0,0.0,0.0,1168.233561,0.0,0.0,40.206138
704,0,21700,20627.0,52040.0,26.0,0.0,2.0,199.985432,0.0,25.0,27.117690
1723,1,18000,48724.0,72654.0,7.0,0.0,0.0,62.088606,0.0,4.0,31.570299
1261,0,13500,48832.0,144065.0,1.0,0.0,0.0,268.586800,4.0,43.0,42.692397
882,0,11300,16115.0,69113.0,22.0,0.0,2.0,403.515000,0.0,12.0,22.168245
543,0,6700,58741.0,68225.0,9.0,0.0,3.0,173.574416,1.0,31.0,23.587013
1618,1,10300,70147.0,122124.0,1.0,0.0,0.0,58.895686,0.0,7.0,57.995019
859,0,10800,44355.0,57742.0,20.0,0.0,1.0,129.731992,0.0,22.0,38.042768


## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**

1. El conjunto de datos trata de una serie de préstamos otorgados por un Banco. Se dispone de la información de los distintos préstamos otorgados, clasificados como incumplidos o reembolsados, y los determinantes que explican ese incumplimiento o reembolso.
2. La variable objetivo a predecir es la variable binaria "TARGET" que toma valor 1 en caso de que el préstamo sea incumplido y 0 si existe reembolso del mismo.
3. Los atributos para determinar si existe incumplimiento o reembolso para cada uno de los préstamos (diferentes clientes) son:
- LOAN: monto del préstamo solicitado
- MORTDUE: monto adeudado de la hipoteca existente
- VALUE: valor de la propiedad actual
- YOJ: años en el trabajo actual
- DEROG: número de informes despectivos importantes
- DELINQ: número de lineas de crédito morosas
- CLAGE: edad de la línea comercial más antigua en meses
- NINQ Número de líneas de crédito recientes
- CLNO Número de líneas de crédito
- DEBTINC Relación deuda-ingresos
4. Los atributos posiblemente más importantes son DELINQ y DEBTINC

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


In [7]:
# Estandarizar atributos para mejorar resultados
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [8]:
# Entrenamiento
sgd_model_base = SGDClassifier(random_state=0)
sgd_model_base.fit(X_train, y_train);

In [9]:
# Predicciones
y_train_pred_sgd_base = sgd_model_base.predict(X_train)
y_test_pred_sgd_base = sgd_model_base.predict(X_test)

#### Métricas sobre conjunto de entrenamiento

In [10]:
metrics_sgd_base_train = get_metrics(y_train, y_train_pred_sgd_base)

metrics_sgd_base_train

,Accuracy,Precision,Recall,F1-Score
0,87.32%,71.43%,41.83%,52.76%


In [11]:
get_confusion_matrix(y_train, y_train_pred_sgd_base)

,Actual - Repaid,Actual - Default,Total
Predicted - Repaid,1190,146,1336
Predicted - Default,42,105,147
Total,1232,251,1483


#### Métricas sobre conjunto de evaluación

In [12]:
metrics_sgd_base_test = get_metrics(y_test, y_test_pred_sgd_base)

metrics_sgd_base_test

,Accuracy,Precision,Recall,F1-Score
0,86.79%,64.52%,34.48%,44.94%


In [13]:
get_confusion_matrix(y_test, y_test_pred_sgd_base)

,Actual - Repaid,Actual - Default,Total
Predicted - Repaid,302,38,340
Predicted - Default,11,20,31
Total,313,58,371


### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [14]:
# Lista de diferentes valores para los parámetros
# Hacemos dos diccionarios de parámetros porque learning_rate='optimal' acepta eta0=0.0, únicamente
param_grid_sgd = [
    {
        'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
        'penalty': ['l2', 'l1', 'elasticnet'],
        'learning_rate': ['constant', 'invscaling', 'adaptive'],
        'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3],
        'eta0': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
    },
    {
        'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
        'penalty': ['l2', 'l1', 'elasticnet'],
        'learning_rate': ['optimal'],
        'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3],
    }
]

In [15]:
sgd_model_gscv = SGDClassifier(random_state=0, max_iter=2000)

sgd_cv = GridSearchCV(sgd_model_gscv, param_grid_sgd, scoring='accuracy', cv=5)
sgd_cv.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=2000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l2', power_t=0.5, random_state=0,
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, ver...
                          'loss': ['hinge', 'log', 'modified_huber',
                                   'squared_hinge', 'perceptron'],
                          'penalty': ['l2', 'l1', 'elasticnet']},
                         {'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                  

In [16]:
results_sgd = pd.DataFrame(sgd_cv.cv_results_)
results_sgd = results_sgd.sort_values(by=['rank_test_score'], ascending=True).reset_index(drop=True)

print('Resultados de GridSearch Cross-Validation:\n')
results_sgd.head(5)

Resultados de GridSearch Cross-Validation:



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_eta0,param_learning_rate,param_loss,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006420,0.000449,0.000617,0.000010,0.01,1,adaptive,hinge,l2,"{'alpha': 0.01, 'eta0': 1.0, 'learning_rate': ...",0.882155,0.861953,0.865320,0.881757,0.864865,0.871210,0.008851,1
1,0.012436,0.000405,0.000669,0.000026,0.001,100,adaptive,hinge,l1,"{'alpha': 0.001, 'eta0': 100.0, 'learning_rate...",0.882155,0.861953,0.868687,0.881757,0.861486,0.871208,0.009139,2
2,0.010752,0.001066,0.000675,0.000034,0.001,1,adaptive,hinge,elasticnet,"{'alpha': 0.001, 'eta0': 1.0, 'learning_rate':...",0.882155,0.861953,0.868687,0.881757,0.861486,0.871208,0.009139,2
3,0.010861,0.001080,0.000763,0.000093,0.001,1,adaptive,hinge,l1,"{'alpha': 0.001, 'eta0': 1.0, 'learning_rate':...",0.882155,0.861953,0.868687,0.881757,0.861486,0.871208,0.009139,2
4,0.020206,0.002100,0.000691,0.000031,0.001,1,adaptive,log,elasticnet,"{'alpha': 0.001, 'eta0': 1.0, 'learning_rate':...",0.865320,0.868687,0.868687,0.891892,0.858108,0.870539,0.011354,5


In [17]:
best_params_sgd = results_sgd.loc[0, 'params']

print('Mejores parámetros:\n')
best_params_sgd

Mejores parámetros:



{'alpha': 0.01,
 'eta0': 1.0,
 'learning_rate': 'adaptive',
 'loss': 'hinge',
 'penalty': 'l2'}

In [18]:
# Entrenamiento
sgd_model_opt = SGDClassifier(**best_params_sgd, random_state=0)
sgd_model_opt.fit(X_train, y_train);

In [19]:
# Predicciones
y_train_pred_sgd_opt = sgd_model_opt.predict(X_train)
y_test_pred_sgd_opt = sgd_model_opt.predict(X_test)

#### Métricas sobre conjunto de entrenamiento

In [20]:
metrics_sgd_opt_train = get_metrics(y_train, y_train_pred_sgd_opt)

metrics_sgd_opt_train

,Accuracy,Precision,Recall,F1-Score
0,87.32%,92.00%,27.49%,42.33%


In [21]:
get_confusion_matrix(y_train, y_train_pred_sgd_opt)

,Actual - Repaid,Actual - Default,Total
Predicted - Repaid,1226,182,1408
Predicted - Default,6,69,75
Total,1232,251,1483


#### Métricas sobre conjunto de evaluación

In [22]:
metrics_sgd_opt_test = get_metrics(y_test, y_test_pred_sgd_opt)

metrics_sgd_opt_test

,Accuracy,Precision,Recall,F1-Score
0,88.41%,94.12%,27.59%,42.67%


In [23]:
get_confusion_matrix(y_test, y_test_pred_sgd_opt)

,Actual - Repaid,Actual - Default,Total
Predicted - Repaid,312,42,354
Predicted - Default,1,16,17
Total,313,58,371


#### Resumen

In [24]:
df_sgd_summary = pd.concat(
    [metrics_sgd_base_train, metrics_sgd_base_test, metrics_sgd_opt_train, metrics_sgd_opt_test]
)
df_sgd_summary.index = ['SGD Base - Train', 'SGD Base - Test', 'SGD Optimized - Train', 'SGD Optimized - Test']

print('Resumen:\n')
df_sgd_summary

Resumen:



,Accuracy,Precision,Recall,F1-Score
SGD Base - Train,87.32%,71.43%,41.83%,52.76%
SGD Base - Test,86.79%,64.52%,34.48%,44.94%
SGD Optimized - Train,87.32%,92.00%,27.49%,42.33%
SGD Optimized - Test,88.41%,94.12%,27.59%,42.67%


## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html


### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


In [25]:
# Entrenamiento
clf = tree.DecisionTreeClassifier(random_state=0)
# dt_model_base = tree.DecisionTreeClassifier(random_state=0,)
dt_model_base = clf.fit(X_train, y_train)

In [26]:
# Predicciones
y_train_pred_dt_base = dt_model_base.predict(X_train)
y_test_pred_dt_base = dt_model_base.predict(X_test)

In [27]:
# Metrics
metrics_dt_base_train = get_metrics(y_train, y_train_pred_dt_base)
metrics_dt_base_train

,Accuracy,Precision,Recall,F1-Score
0,100.00%,100.00%,100.00%,100.00%


In [28]:
# Confusion matrix
get_confusion_matrix(y_train, y_train_pred_dt_base)

,Actual - Repaid,Actual - Default,Total
Predicted - Repaid,1232,0,1232
Predicted - Default,0,251,251
Total,1232,251,1483


### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [29]:
# Lista de diferentes valores para los parámetros

param_grid_dt = [
    {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(1, 30),
        'min_samples_leaf': [1, 5, 10, 20, 50, 100]
    }
]

In [30]:
dt_model_gscv = tree.DecisionTreeClassifier(random_state=0)

dt_cv = GridSearchCV(dt_model_gscv, param_grid_dt, scoring='accuracy', cv=5)
dt_cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': array([ 1,  2,  3,  4,  5,  

In [31]:
results_dt = pd.DataFrame(dt_cv.cv_results_)
results_dt = results_dt.sort_values(by=['rank_test_score'], ascending=True).reset_index(drop=True)

print('Resultados de GridSearch Cross-Validation:\n')
results_dt.head(5)

Resultados de GridSearch Cross-Validation:



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006295,0.000071,0.000582,0.000016,gini,7,10,"{'criterion': 'gini', 'max_depth': 7, 'min_sam...",0.858586,0.882155,0.895623,0.891892,0.888514,0.883354,0.013152,1
1,0.005904,0.000147,0.000674,0.000052,gini,6,10,"{'criterion': 'gini', 'max_depth': 6, 'min_sam...",0.878788,0.882155,0.892256,0.885135,0.878378,0.883342,0.005088,2
2,0.007653,0.000140,0.000593,0.000010,gini,9,1,"{'criterion': 'gini', 'max_depth': 9, 'min_sam...",0.845118,0.875421,0.895623,0.912162,0.878378,0.881340,0.022405,3
3,0.008295,0.000319,0.000640,0.000045,gini,10,5,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.851852,0.878788,0.878788,0.915541,0.878378,0.880669,0.020292,4
4,0.008016,0.000178,0.000709,0.000050,entropy,5,10,"{'criterion': 'entropy', 'max_depth': 5, 'min_...",0.885522,0.875421,0.882155,0.881757,0.878378,0.880647,0.003456,5


In [32]:
# Mejores params

best_params_dt = results_dt.loc[0, 'params']

print('Mejores parámetros:\n')
best_params_dt

Mejores parámetros:



{'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 10}

In [33]:
# Entrenamiento
dt_model_opt = tree.DecisionTreeClassifier(**best_params_dt, random_state=0)
dt_model_opt.fit(X_train, y_train);

In [34]:
# Predicciones
y_train_pred_dt_opt = dt_model_opt.predict(X_train)
y_test_pred_dt_opt = dt_model_opt.predict(X_test)

#### Métricas sobre conjunto de entrenamiento

In [35]:
metrics_dt_opt_train = get_metrics(y_train, y_train_pred_dt_opt)

metrics_dt_opt_train

,Accuracy,Precision,Recall,F1-Score
0,89.75%,90.24%,44.22%,59.36%


In [36]:
get_confusion_matrix(y_train, y_train_pred_dt_opt)

,Actual - Repaid,Actual - Default,Total
Predicted - Repaid,1220,140,1360
Predicted - Default,12,111,123
Total,1232,251,1483


#### Métricas sobre conjunto de evaluación

In [37]:
metrics_dt_opt_test = get_metrics(y_test, y_test_pred_dt_opt)

metrics_dt_opt_test

,Accuracy,Precision,Recall,F1-Score
0,89.22%,80.00%,41.38%,54.55%


In [38]:
get_confusion_matrix(y_test, y_test_pred_dt_opt)

,Actual - Repaid,Actual - Default,Total
Predicted - Repaid,307,34,341
Predicted - Default,6,24,30
Total,313,58,371
